In [1]:
import json
import os
from IPython.display import Image
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
def load_json(path):
    with open(path) as f:
        data = [json.loads(line) for line in f]
    return data

def load_json_file(path):
    with open(path, 'r', encoding='utf8') as f:
        return json.load(f)

def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [3]:
#%cd '/content/drive/MyDrive/AdaptOrDie/dont-stop-pretraining-modified'
%cd 'dont-stop-pretraining-modified'

/home/jupyter/dental-notebooks/ltuzova/gatech_dl/final/AdaptOrDie/dont-stop-pretraining-modified


In [4]:
#Path to virtual environment on my work server. Don't need it if run under colab
venv_path = "/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/bin"  # Adjust this path as necessary
os.environ["PATH"] = venv_path + os.pathsep + os.environ["PATH"]

# Download Pre-train models

## DAPT 25M

In [ ]:
# !python -m scripts.download_model \
#         --model ltuzova/amazon_helpfulness_classification_on_25M_full_pretrained \
#         --serialization_dir $(pwd)/pretrained_models/amazon_helpfulness_classification_on_25M_full_pretrained

## DAPT 5M

In [ ]:
# !python -m scripts.download_model \
#         --model ltuzova/amazon_helpfulness_classification_on_5M_full_pretrained \
#         --serialization_dir $(pwd)/pretrained_models/amazon_helpfulness_classification_on_5M_full_pretrained

## TAPT (20 epochs)

In [ ]:
# !python -m scripts.download_model \
#         --model ltuzova/tapt_helpfulness_base_pretraining_model_full_train \
#         --serialization_dir $(pwd)/pretrained_models/amazon_helpfulness_classification_on_tapt_full_pretrained

## TAPT (100 epochs, test as train)

In [7]:
!python -m scripts.download_model \
        --model ltuzova/tapt_helpfulness_base_pretraining_model \
        --serialization_dir $(pwd)/pretrained_models/amazon_helpfulness_classification_on_tapt_test_as_train

Some weights of the model checkpoint at ltuzova/tapt_helpfulness_base_pretraining_model were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ltuzova/tapt_helpfulness_base_pretraining_model and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to 

## DAPT/TAPT

In [24]:
# !python -m scripts.download_model \
#         --model ltuzova/dapt_plus_tapt_helpfulness_base_pretraining_model \
#         --serialization_dir $(pwd)/pretrained_models/amazon_helpfulness_classification_on_dapt_tapt

## Run Amazon Classification on 25M DAPT pretrained model

In [ ]:
!python -m scripts.train \
        --config training_config/classifier.jsonnet \
        --serialization_dir model_logs/amazon_dapt_25_1 \
        --hyperparameters ROBERTA_CLASSIFIER_SMALL \
        --dataset amazon \
        --model $(pwd)/pretrained_models/amazon_helpfulness_classification_on_25M_full_pretrained \
        --device 3 \
        --perf +f1 \
        --evaluate_on_test

/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")
2024-04-22 23:11:30,818 - INFO - allennlp.common.params - evaluation = None
2024-04-22 23:11:30,818 - INFO - allennlp.common.params - include_in_archive = None
2024-04-22 23:11:30,818 - INFO - allennlp.common.params - random_seed = 343511
2024-04-22 23:11:30,818 - INFO - allennlp.common.params - numpy_seed = 343511
2024-04-22 23:11:30,818 - INFO - allennlp.common.params - pytorch_seed = 343511
2024-04-22 23:11:30,819 - INFO - allennlp.common.checks - Pytorch version: 1.13.1+cu117
2024-04-22 23:11:30

In [1]:
print("Complete")

Complete


In [ ]:
m = [{
    'exp':'dapt_25_1',

}]

In [21]:
m_json = load_json_file("model_logs/amazon_dapt_25_1/metrics.json")
m_json['exp']='dapt_25_1'

metrics = pd.DataFrame([m_json])

metrics

,best_epoch,peak_worker_0_memory_MB,peak_gpu_0_memory_MB,training_duration,epoch,training_f1,training_accuracy,training_loss,training_worker_0_memory_MB,training_gpu_0_memory_MB,validation_f1,validation_accuracy,validation_loss,best_validation_f1,best_validation_accuracy,best_validation_loss,test_f1,test_accuracy,test_loss,exp
0,6,11394.535156,0,5:45:20.509730,9,0.959714,0.980078,0.062252,11394.535156,0.0,0.656391,0.8648,0.518977,0.705725,0.866,0.534225,0.689808,0.86116,0.543773,dapt_25_1


## Run Amazon Classification on 5M DAPT pretrained model

In [12]:
!python -m scripts.train \
        --config training_config/classifier.jsonnet \
        --serialization_dir model_logs/amazon_dapt_5_1 \
        --hyperparameters ROBERTA_CLASSIFIER_SMALL \
        --dataset amazon \
        --model $(pwd)/pretrained_models/amazon_helpfulness_classification_on_5M_full_pretrained \
        --device 3 \
        --perf +f1 \
        --evaluate_on_test

/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")
2024-04-23 05:27:57,673 - INFO - allennlp.common.params - evaluation = None
2024-04-23 05:27:57,673 - INFO - allennlp.common.params - include_in_archive = None
2024-04-23 05:27:57,674 - INFO - allennlp.common.params - random_seed = 524145
2024-04-23 05:27:57,674 - INFO - allennlp.common.params - numpy_seed = 524145
2024-04-23 05:27:57,674 - INFO - allennlp.common.params - pytorch_seed = 524145
2024-04-23 05:27:57,675 - INFO - allennlp.common.checks - Pytorch version: 1.13.1+cu117
2024-04-23 05:27:57

In [18]:
m = [{
    'exp':'dapt_5_1',
    "best_epoch": 0,
  "peak_worker_0_memory_MB": 11396.48046875,
  "peak_gpu_0_memory_MB": 0,
  "training_duration": "2:17:30.851626",
  "epoch": 3,
  "training_f1": 0.8819594085216522,
  "training_accuracy": 0.9445731490399215,
  "training_loss": 0.15612382236064032,
  "training_worker_0_memory_MB": 11396.48046875,
  "training_gpu_0_memory_MB": 0.0,
  "validation_f1": 0.6469497531652451,
  "validation_accuracy": 0.8674,
  "validation_loss": 0.4431749629741981,
  "best_validation_f1": 0.6848366856575012,
  "best_validation_accuracy": 0.8756,
  "best_validation_loss": 0.3148426094863076,
  "test_f1": 0.6826630979776382,
  "test_accuracy": 0.87704,
  "test_loss": 0.30553998724365,
}]

In [19]:
metrics = pd.DataFrame(m)

In [20]:
metrics

,exp,best_epoch,peak_worker_0_memory_MB,peak_gpu_0_memory_MB,training_duration,epoch,training_f1,training_accuracy,training_loss,training_worker_0_memory_MB,training_gpu_0_memory_MB,validation_f1,validation_accuracy,validation_loss,best_validation_f1,best_validation_accuracy,best_validation_loss,test_f1,test_accuracy,test_loss
0,dapt_5_1,0,11396.480469,0,2:17:30.851626,3,0.881959,0.944573,0.156124,11396.480469,0.0,0.64695,0.8674,0.443175,0.684837,0.8756,0.314843,0.682663,0.87704,0.30554


## Run Amazon Classification on TAPT pretrained model

In [14]:
!python -m scripts.train \
        --config training_config/classifier.jsonnet \
        --serialization_dir model_logs/amazon_tapt_1 \
        --hyperparameters ROBERTA_CLASSIFIER_SMALL \
        --dataset amazon \
        --model $(pwd)/pretrained_models/amazon_helpfulness_classification_on_tapt_full_pretrained \
        --device 3 \
        --perf +f1 \
        --evaluate_on_test

/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")
2024-04-23 07:51:58,102 - INFO - allennlp.common.params - evaluation = None
2024-04-23 07:51:58,102 - INFO - allennlp.common.params - include_in_archive = None
2024-04-23 07:51:58,102 - INFO - allennlp.common.params - random_seed = 203752
2024-04-23 07:51:58,102 - INFO - allennlp.common.params - numpy_seed = 203752
2024-04-23 07:51:58,103 - INFO - allennlp.common.params - pytorch_seed = 203752
2024-04-23 07:51:58,104 - INFO - allennlp.common.checks - Pytorch version: 1.13.1+cu117
2024-04-23 07:51:58

In [15]:
m = [{
    'exp':'tapt_1',
    "best_epoch": 4,
  "peak_worker_0_memory_MB": 11389.94140625,
  "peak_gpu_0_memory_MB": 0,
  "training_duration": "4:35:15.788292",
  "epoch": 7,
  "training_f1": 0.953032523393631,
  "training_accuracy": 0.9767811125283078,
  "training_loss": 0.07056242575980984,
  "training_worker_0_memory_MB": 11389.94140625,
  "training_gpu_0_memory_MB": 0.0,
  "validation_f1": 0.662454679608345,
  "validation_accuracy": 0.8694,
  "validation_loss": 0.5573178373065929,
  "best_validation_f1": 0.7047898471355438,
  "best_validation_accuracy": 0.8712,
  "best_validation_loss": 0.41303316756678277,
  "test_f1": 0.7009774595499039,
  "test_accuracy": 0.87264,
  "test_loss": 0.41956981175244856,
}]

In [16]:
metrics = pd.DataFrame(m)

In [17]:
metrics

,exp,best_epoch,peak_worker_0_memory_MB,peak_gpu_0_memory_MB,training_duration,epoch,training_f1,training_accuracy,training_loss,training_worker_0_memory_MB,training_gpu_0_memory_MB,validation_f1,validation_accuracy,validation_loss,best_validation_f1,best_validation_accuracy,best_validation_loss,test_f1,test_accuracy,test_loss
0,tapt_1,4,11389.941406,0,4:35:15.788292,7,0.953033,0.976781,0.070562,11389.941406,0.0,0.662455,0.8694,0.557318,0.70479,0.8712,0.413033,0.700977,0.87264,0.41957


## Run Amazon Classification on DAPT/TAPT

In [ ]:
!python -m scripts.train \
        --config training_config/classifier.jsonnet \
        --serialization_dir model_logs/amazon_dapt_tapt_1 \
        --hyperparameters ROBERTA_CLASSIFIER_SMALL \
        --dataset amazon \
        --model $(pwd)/pretrained_models/amazon_helpfulness_classification_on_25M_full_pretrained \
        --device 3 \
        --perf +f1 \
        --evaluate_on_test

/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")
2024-04-23 16:24:44,438 - INFO - allennlp.common.params - evaluation = None
2024-04-23 16:24:44,438 - INFO - allennlp.common.params - include_in_archive = None
2024-04-23 16:24:44,440 - INFO - allennlp.common.params - random_seed = 375331
2024-04-23 16:24:44,440 - INFO - allennlp.common.params - numpy_seed = 375331
2024-04-23 16:24:44,440 - INFO - allennlp.common.params - pytorch_seed = 375331
2024-04-23 16:24:44,441 - INFO - allennlp.common.checks - Pytorch version: 1.13.1+cu117
2024-04-23 16:24:44

In [6]:
m_json = load_json_file("model_logs/amazon_dapt_tapt_1/metrics.json")
m_json['exp']='dapt_tapt_1'

metrics = pd.DataFrame([m_json])

metrics

,best_epoch,peak_worker_0_memory_MB,peak_gpu_0_memory_MB,training_duration,epoch,training_f1,training_accuracy,training_loss,training_worker_0_memory_MB,training_gpu_0_memory_MB,validation_f1,validation_accuracy,validation_loss,best_validation_f1,best_validation_accuracy,best_validation_loss,test_f1,test_accuracy,test_loss,exp
0,1,11388.835938,0,2:52:15.262381,4,0.910549,0.957076,0.125132,11388.835938,0.0,0.648473,0.8702,0.461334,0.669109,0.8752,0.31749,0.665298,0.8772,0.313594,dapt_tapt_1


## Run Amazon Classification on TAPT, 100 epohcs, test as train

In [8]:
!python -m scripts.train \
        --config training_config/classifier.jsonnet \
        --serialization_dir model_logs/amazon_tapt_test_1 \
        --hyperparameters ROBERTA_CLASSIFIER_SMALL \
        --dataset amazon \
        --model $(pwd)/pretrained_models/amazon_helpfulness_classification_on_tapt_test_as_train \
        --device 3 \
        --perf +f1 \
        --evaluate_on_test

/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")
2024-04-23 21:07:48,548 - INFO - allennlp.common.params - evaluation = None
2024-04-23 21:07:48,548 - INFO - allennlp.common.params - include_in_archive = None
2024-04-23 21:07:48,548 - INFO - allennlp.common.params - random_seed = 33377
2024-04-23 21:07:48,549 - INFO - allennlp.common.params - numpy_seed = 33377
2024-04-23 21:07:48,549 - INFO - allennlp.common.params - pytorch_seed = 33377
2024-04-23 21:07:48,549 - INFO - allennlp.common.checks - Pytorch version: 1.13.1+cu117
2024-04-23 21:07:48,55

In [10]:
m_json = load_json_file("model_logs/amazon_tapt_test_1/metrics.json")
m_json['exp']='tapt_test_1'

metrics = pd.DataFrame([m_json])

metrics

,best_epoch,peak_worker_0_memory_MB,peak_gpu_0_memory_MB,training_duration,epoch,training_f1,training_accuracy,training_loss,training_worker_0_memory_MB,training_gpu_0_memory_MB,validation_f1,validation_accuracy,validation_loss,best_validation_f1,best_validation_accuracy,best_validation_loss,test_f1,test_accuracy,test_loss,exp
0,2,11396.84375,0,3:27:40.032650,5,0.90797,0.955801,0.127602,11396.84375,0.0,0.690396,0.8696,0.421514,0.695158,0.873,0.336145,0.694175,0.87548,0.338853,tapt_test_1
